In [7]:
from torchvision import models
from PIL import Image


resnet_weigths = models.ResNet50_Weights.DEFAULT 
cats = resnet_weigths.meta['categories']
transforms = resnet_weigths.transforms()


model = models.resnet50(weights=resnet_weigths)

img= Image.open("datasets\\resnet\\Nokota_Horses_cropped.jpg").convert('RGB')
img_t = transforms(img).unsqueeze(0)

model.eval()
p = model(img_t).squeeze()

res = p.softmax(dim=0).sort(descending=True)

for s, i in zip(res[0][:5], res[1][:5]):
    print(f"{cats[i]}:{s:.4f}")


# print(model)

sorrel:0.0581
horse cart:0.0564
hartebeest:0.0498
warthog:0.0181
plow:0.0147


In [ ]:
# 2
# в каждом списке 4 значения - количество блоков (по порядку)
# в слоях Layer1, Layer2, Layer3 и Layer4
resnet18 = [2, 2, 2, 2]
resnet34 = [3, 4, 6, 3]
resnet50 = [3, 4, 6, 3]

In [12]:
# 3
import torch
import torch.nn as nn

# здесь объявляйте классы BasicBlock1 и BasicBlock2

batch_size = 8
x = torch.rand(batch_size, 3, 32, 32) # тензор x в программе не менять

# здесь объявляйте классы BasicBlock1 и BasicBlock2
# здесь продолжайте программу
class BasicBlock1(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.basic_block = nn.Sequential(
            nn.Conv2d(64, 128, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(128),
        )

        self.skip_conn = nn.Sequential(
            nn.Conv2d(64, 128, 1, stride=2, padding=0, bias=False),
            nn.BatchNorm2d(128)
        )

    def forward(self, x):
        f = self.basic_block(x)
        sk = self.skip_conn(x)
        return (f + sk).relu()
    
class BasicBlock2(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.basic_block = nn.Sequential(
            nn.Conv2d(128, 128, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(128),
        )

    def forward(self, x):
        f = self.basic_block(x)
        return (f+x).relu()

model = nn.Sequential(
    nn.Conv2d(3, 64, 7, stride=2, padding=3, bias=False),
    nn.BatchNorm2d(64),
    nn.MaxPool2d(3, stride=2, padding=1),
    BasicBlock1(),
    BasicBlock2(),
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(128, 10)
)

model.eval()
predict = model(x)

In [ ]:
# 4 
import torch
import torch.nn as nn

# здесь объявляйте классы BottleneckBlock1 и BottleneckBlock2
class BottleneckBlock1(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.bn = nn.Sequential(
            nn.Conv2d(64,64, 1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3 ,stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64*4, 1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(64*4)
        )
        self.sk = nn.Sequential(
            nn.Conv2d(64, 64*4, 1, stride=1, padding=0,bias=False),
            nn.BatchNorm2d(64*4)
        )

    def forward(self,x):
        bn = self.bn(x)
        sk = self.sk(x)
        return (bn + sk).relu()
    

class BottleneckBlock2(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.bn = nn.Sequential(
            nn.Conv2d(64*4, 64, 1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3 ,stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64*4, 1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(64*4)
        )

    def forward(self, x):
        bn = self.bn(x)
        return (bn + x).relu()

batch_size = 8
x = torch.rand(batch_size, 3, 32, 32) # тензор x в программе не менять

# здесь продолжайте программу
model = nn.Sequential(
    nn.Conv2d(3, 64, 7, stride=2, padding=3, bias=False),
    nn.BatchNorm2d(64),
    nn.MaxPool2d(3, stride=2, padding=1),
    BottleneckBlock1(),
    BottleneckBlock2(),
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(64*4, 10, bias=True)
)

model.eval()
predict = model(x)